<a href="https://colab.research.google.com/github/Sumetjutha/MyProject/blob/main/5_Build_A_Stock_Prediction_TMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quandl

In [ ]:
# This program Predicts stock prices by nusing machine learning models
# Install the dependencies
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import pandas as pd
from pandas_datareader import data
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = data.DataReader('TMB.bk',data_source="yahoo")
df.to_csv('/content/drive/MyDrive/MyProject/TMB_BK.csv')
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-04,2.19477,2.13990,2.19477,2.13990,81077240.0,1.835067
2016-01-05,2.15819,2.10332,2.15819,2.12161,106283497.0,1.819382
2016-01-06,2.19477,2.12161,2.13990,2.17648,102702474.0,1.866436
2016-01-07,2.15819,2.12161,2.12161,2.12161,83287987.0,1.819382
2016-01-08,2.15819,2.12161,2.13990,2.15819,92758869.0,1.850752


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MyProject/TMB_BK_adjust.csv") 
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,04-01-16,2.19477,2.13990,2.19477,2.13990,81077240,1.835067
1,05-01-16,2.15819,2.10332,2.15819,2.12161,106283497,1.819382
2,06-01-16,2.19477,2.12161,2.13990,2.17648,102702474,1.866436
3,07-01-16,2.15819,2.12161,2.12161,2.12161,83287987,1.819382
4,08-01-16,2.15819,2.12161,2.13990,2.15819,92758869,1.850752


In [ ]:
# Get the Adjusted Close Price
df = df[['Adj Close']]

In [ ]:
# Take a look at the new data
print(df.head())

   Adj Close
0   1.835067
1   1.819382
2   1.866436
3   1.819382
4   1.850752


In [ ]:
print(df.tail())

      Adj Close
1194       1.13
1195       1.16
1196       1.15
1197       1.13
1198       1.05


In [ ]:
# A variable for prediction 'n' days out into the future
forecast_out = 19

In [ ]:
# Create anothr column (the target or dependent variable) shifted 'n' units up
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)

In [ ]:
print(df.head())

   Adj Close  Prediction
0   1.835067    2.023282
1   1.819382    2.007597
2   1.866436    1.991912
3   1.819382    1.991912
4   1.850752    2.038966


In [ ]:
print(df.tail())

      Adj Close  Prediction
1194       1.13         NaN
1195       1.16         NaN
1196       1.15         NaN
1197       1.13         NaN
1198       1.05         NaN


In [ ]:
### Create the Independent data set (X)
# convert the data frame to a numpy array
X = np.array(df.drop(['Prediction'],1))

In [ ]:
# Remove the last 'n' rows 
X = X[:-forecast_out]

In [ ]:
print(X)

[[1.83506692]
 [1.81938207]
 [1.86643624]
 ...
 [0.81      ]
 [0.81999999]
 [0.83999997]]


In [ ]:
### Create the independent data set (y)
# Convert the dataframe to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])

In [ ]:
# Get all of the y values except the last 'n' rows
y = y[:-forecast_out]

In [ ]:
print(y)

[2.02328181 2.00759721 1.99191248 ... 1.14999998 1.12999999 1.04999995]


In [ ]:
# Split the data into 80% training and 20% testting
x_train,x_test,y_train,y_test = train_test_split(X, y ,test_size=0.5)

In [ ]:
# Create and train the Support Vector Machine (Regressor)
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)

In [ ]:
svr_rbf.fit(x_train,y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test,y_test)

In [ ]:
print("svm confidence: ", svm_confidence)

svm confidence:  0.8962803306550702


In [ ]:
# Create and trainLinear Regression Model
lr = LinearRegression()

In [ ]:
# Train the model
lr.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# Testing Model: Score returns the coefficient of determination R^2 of the prediction
# The best possible score is 1.0
lr_confidence = lr.score(x_test,y_test)

In [ ]:
print("lr confidence: ", lr_confidence)

lr confidence:  0.8899031824551087


In [ ]:
# Set x_forecast equal to the last 30 rows of the original ata set from Adj Close columns
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]

In [ ]:
print(x_forecast)

[[0.85000002]
 [0.88999999]
 [0.92000002]
 [0.93000001]
 [1.02999997]
 [0.99000001]
 [0.98000002]
 [1.01999998]
 [1.01999998]
 [0.99000001]
 [0.98000002]
 [1.        ]
 [1.04999995]
 [1.15999997]
 [1.12999999]
 [1.15999997]
 [1.14999998]
 [1.12999999]
 [1.04999995]]


In [ ]:
# Print the predictions for the next 'n' days
lr_prediction = lr.predict(x_forecast)

In [ ]:
print(lr_prediction)

[0.86534572 0.9041531  0.93325869 0.94296054 1.03997905 1.00117167
 0.99146983 1.03027721 1.03027721 1.00117167 0.99146983 1.01087352
 1.05938274 1.16610316 1.13699762 1.16610316 1.15640132 1.13699762
 1.05938274]


In [ ]:
# Print the predictions for the next 'n' days
svm_prediction = svr_rbf.predict(x_forecast)

In [ ]:
print(svm_prediction)

[0.91998604 0.94065172 0.95772966 0.96371167 1.03093878 1.00249226
 0.99569573 1.02364242 1.02364242 1.00249226 0.99569573 1.009417
 1.04588792 1.13588065 1.11011923 1.13588065 1.12719919 1.11011923
 1.04588792]
